In [16]:
from argparse import ArgumentParser
from mmdet3d.apis import inference_mono_3d_detector, init_model, show_result_meshlab
import re
from copy import deepcopy
from os import path as osp
import open3d as o3d
from open3d.web_visualizer import draw
import sys
import os
from mmdet3d.apis import inference_detector, init_model, show_result_meshlab
import mmcv
import numpy as np
import torch
from mmcv.parallel import collate, scatter
from mmcv.runner import load_checkpoint

from mmdet3d.core import (Box3DMode, CameraInstance3DBoxes, Coord3DMode,
                          DepthInstance3DBoxes, LiDARInstance3DBoxes,
                          show_multi_modality_result, show_result,
                          show_seg_result)
from mmdet3d.core.bbox import get_box_type
from mmdet3d.datasets.pipelines import Compose
from mmdet3d.models import build_model
from mmdet3d.utils import get_root_logger

[Open3D INFO] Resetting default logger to print to terminal.


In [2]:
parser = ArgumentParser()
parser.add_argument(
    '-pcd', default='/home/server-003/dl/mmdetection3d/demo/data/kitti/kitti_000008.bin')
parser.add_argument(
    '--config', default='/home/server-003/dl/mmdetection3d/configs/second/hv_second_secfpn_6x8_80e_kitti-3d-car.py')
parser.add_argument(
    '--checkpoint', default='/home/server-003/dl/mmdetection3d/checkpoints/hv_second_secfpn_6x8_80e_kitti-3d-car_20200620_230238-393f000c.pth')
parser.add_argument(
    '--device', default='cuda:0', help='Device used for inference')
parser.add_argument(
    '--score-thr', type=float, default=0.0, help='bbox score threshold')
args = parser.parse_args(args=[])
cfg = mmcv.Config.fromfile(args.config)
print(f'Config:\n{cfg.pretty_text}')

Config:
voxel_size = [0.05, 0.05, 0.1]
model = dict(
    type='VoxelNet',
    voxel_layer=dict(
        max_num_points=5,
        point_cloud_range=[0, -40, -3, 70.4, 40, 1],
        voxel_size=[0.05, 0.05, 0.1],
        max_voxels=(16000, 40000)),
    voxel_encoder=dict(type='HardSimpleVFE'),
    middle_encoder=dict(
        type='SparseEncoder',
        in_channels=4,
        sparse_shape=[41, 1600, 1408],
        order=('conv', 'norm', 'act')),
    backbone=dict(
        type='SECOND',
        in_channels=256,
        layer_nums=[5, 5],
        layer_strides=[1, 2],
        out_channels=[128, 256]),
    neck=dict(
        type='SECONDFPN',
        in_channels=[128, 256],
        upsample_strides=[1, 2],
        out_channels=[256, 256]),
    bbox_head=dict(
        type='Anchor3DHead',
        num_classes=1,
        in_channels=512,
        feat_channels=512,
        use_direction_classifier=True,
        anchor_generator=dict(
            type='Anchor3DRangeGenerator',
            r

In [3]:
model = init_model(args.config, args.checkpoint, device=args.device)

/home/server-003/dl/mmdetection3d/mmdet3d/models/dense_heads/anchor3d_head.py:85: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  'dir_offset and dir_limit_offset will be depressed and be '


load checkpoint from local path: /home/server-003/dl/mmdetection3d/checkpoints/hv_second_secfpn_6x8_80e_kitti-3d-car_20200620_230238-393f000c.pth


In [4]:
def draw_bboxes(bbox3d, pcd=None, bbox_color=(0, 1, 0), points_in_box_color=(1, 0, 0), rot_axis=2, center_mode='lidar_bottom'):
    bbox3d = bbox3d.copy()
    in_box_color = np.array(points_in_box_color)
    list_set = []
    for i in range(len(bbox3d)):
        center = bbox3d[i, 0:3]
        dim = bbox3d[i, 3:6]
        yaw = np.zeros(3)
        yaw[rot_axis] = bbox3d[i, 6]
        rot_mat = o3d.geometry.get_rotation_matrix_from_xyz(yaw)
        if center_mode == 'lidar_bottom':
            center[rot_axis] += dim[
                rot_axis] / 2  # bottom center to gravity center
        elif center_mode == 'camera_bottom':
            center[rot_axis] -= dim[
                rot_axis] / 2  # bottom center to gravity center
        box3d = o3d.geometry.OrientedBoundingBox(center, rot_mat, dim)
        list_set.append(box3d)
        line_set = o3d.geometry.LineSet.create_from_oriented_bounding_box(
            box3d)
        line_set.paint_uniform_color(bbox_color)
        list_set.append(line_set)
    return list_set


In [5]:
def show_det_result_meshlab(data, result, score_thr=0.0):

    points = data['points'][0][0].cpu().numpy()

    if 'pts_bbox' in result[0].keys():
        pred_bboxes = result[0]['pts_bbox']['boxes_3d'].tensor.numpy()
        pred_scores = result[0]['pts_bbox']['scores_3d'].numpy()
    else:
        pred_bboxes = result[0]['boxes_3d'].tensor.numpy()
        pred_scores = result[0]['scores_3d'].numpy()

    # filter out low score bboxes for visualization
    if score_thr > 0:
        inds = pred_scores > score_thr
        pred_bboxes = pred_bboxes[inds]

    return draw_bboxes(bbox3d=pred_bboxes)

In [6]:
def depth_color(val, min_d=0, max_d=120):
    """ 
    print Color(HSV's H value) corresponding to distance(m) 
    close distance = red , far distance = blue
    """
    np.clip(val, 0, max_d, out=val)  # max distance is 120m but usually not usual
    return (((val - min_d) / (max_d - min_d)) * 120).astype(np.uint8)

In [7]:
result, data = inference_detector(model, args.pcd)

In [24]:
points = data['points'][0][0].cpu().numpy()
x = points[:, 0]
y = points[:, 1]
z = points[:, 2]
dist = np.sqrt(x**2+y**2+z**2)
ones = np.ones([points.shape[0],2]).transpose()
ones = ones*255
color=depth_color(dist,0,120)
color=np.array([color])
colors = np.concatenate((color,ones),axis=0)

In [25]:
cloud = o3d.geometry.PointCloud()
cloud.points = o3d.utility.Vector3dVector(points[:,:3])
cloud.colors = o3d.utility.Vector3dVector(colors.transpose())
bbox = show_det_result_meshlab(data, result, args.score_thr)



[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]


In [26]:
bbox.append(cloud)
draw(bbox)
'''
TODO
1. apply different color to points in box
2. check bbox location code
3. diffent color according to distance
'''


[Open3D INFO] Window window_3 created.
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]


WebVisualizer(window_uid='window_3')

'\nTODO\n1. apply different color to points in box\n2. check bbox location code\n'

[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Called HTTP API (custom handshake)] /api/call
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceCandidate
[Open3D INFO] DataChannelObserver::OnStateChange label: ServerDataChannel, state: open, peerid: 0.620811125586143
[Open3D INFO] DataChannelObserver::OnStateChange label: ClientDataChannel, state: open, peerid: 0.620811125586143
[Open3D INFO] Sending init frames to window_3.


[549:039][240405] (stun_port.cc:96): Binding request timed out from 192.168.219.x:58592 (wlp147s0)
